In [1]:
import functools
import numpy as np
from scipy.optimize import fmin_bfgs, fmin_l_bfgs_b

# ***function: iterPageRank
# this function takes initial node probabilities and a 
# transition matrix then use power iteration to find 
# the PageRank of nodes
def iterPageRank(pp, trans):
    ppnew = np.dot(pp, trans)
    while not(np.allclose(pp, ppnew)):
        pp = ppnew
        ppnew = np.dot(pp, trans)
    return ppnew


# ***function: genCopyGraph
# generate an undirected random graph with copyig model
# the generated graph has the property of preferencial attachment
# an edge list is returned
def genCopyGraph(nnodes, alpha):
    # since the copy model starts with a triad, the input number of 
    # nodes should be larger than 3
    if nnodes <= 3:
        print "Number of nodes should be larger than 3..."
        return
    # inital setting with a triad
    degrees = np.repeat(2, 3)
    edges = [(0, 1), (0, 2), (1, 2)]
    # growing the graph node by node
    for i in range(3, nnodes):
        # add three edges for the new node
        for nedge in range(3):
            if np.random.rand() < alpha:
                # uniformly choose node to connect
                tar = np.random.choice(i, 1)[0]
                while (tar, i) in edges:
                    tar = np.random.choice(i, 1)[0]
                edges.append((tar, i))
                degrees[tar] += 1
            else:
                # select target to connect according to degree
                sDeg = sum(degrees)
                tar = -1
                firstRound = True
                while (tar, i) in edges or firstRound:
                    firstRound = False
                    randPick = np.random.randint(1, sDeg+1)
                    accu = 0
                    for j in range(len(degrees)):
                        accu += degrees[j]
                        if randPick <= accu:
                            tar = j
                            break
                        
                edges.append((tar, i))
                degrees[tar] += 1
        
        degrees = np.append(degrees, 3)
    return [edges, degrees]


# ***function: calStrength
# return edge strength calculated by logistic function
# the inputs are two vectors, features and the parameters
def calStrength(features, beta):
    #return np.exp(np.dot(features, beta))
	# use logistic strength function to prevent potential overflow or under flow
	# of floating point numbers
	return  1.0 / (1+ np.exp(-1 * np.dot(features, beta) ))


# ***function: strengthDiff
# calculate an returns the gradient of strength functioin with 
# respect to beta, the returned value is a vector
def strengthDiff(features, beta):
    # return a vector of gradient of strength
    diff = []
    denom = calStrength(features, beta) ** 2
    numer_exp = np.exp(-1 * np.dot(features, beta))
    for k in range(len(beta)):
        diff.append(features[k] * numer_exp * denom)
    return diff


# ***function: genTrans
# this function takes in a graph (edge list and number of nodes), 
# node features, source node, and alpha/beta parameters to generate
# a random walk transition matrix.
# transition probabilities are determined by edge strength
# beta is the parameter in the edge strength function
# alpha is the teleportation rate back to the source node
def genTrans(nnodes, g, features, s, alpha, beta):
    # feature is supplied in per-edge manner
    # the transition matrix is created with teleportation
    trans = np.zeros((nnodes, nnodes))
    for i in range(len(g)):
        #strength = calStrength(np.asarray(features[g[i][0],])*np.asarray(features[g[i][1],])
        #, beta)
        strength = calStrength(features[g[i][0]][g[i][1]], beta)
        trans[g[i][0], g[i][1]] = strength
        trans[g[i][1], g[i][0]] = strength
    
    # normalize the transition matrix
    for i in range(nnodes):
        tempSum = sum(trans[i,])
        if tempSum > 0:
            trans[i,] = map(lambda x: x/tempSum, trans[i, ])
    
    # create a list of transition matrices for a set of sources
    trans_multi = []    
    
    for si in range(len(s)):
    # create the one matrix
        one = np.zeros((nnodes, nnodes))
        for i in range(nnodes):
            one[i, s[si]] = 1
            
        # combine the regular transition matrix and the one matrix
        trans_multi.append((1-alpha)*trans + alpha*one) 
    
    return trans_multi

# ***function: genTrans_plain
# this function construct transition matrix for random walk
# with unweighted edge strenght, i.e. each eade has strength 1
def genTrans_plain(nnodes, g, s, alpha):
    trans = np.zeros((nnodes, nnodes))
    for i in range(len(g)):
        trans[g[i][0], g[i][1]] = 1
        trans[g[i][1], g[i][0]] = 1
    
    # normalize the transition matrix
    for i in range(nnodes):
        tempSum = sum(trans[i,])
        if tempSum > 0:
            trans[i,] = map(lambda x: x/tempSum, trans[i, ])
    
    # create a list of transition matrices for a set of sources
    trans_multi = []    
    
    for si in range(len(s)):
        # create the one matrix
        one = np.zeros((nnodes, nnodes))
        for i in range(nnodes):
            one[i, s[si]] = 1
            
        # combine the regular transition matrix and the one matrix
        trans_multi.append((1-alpha)*trans + alpha*one)
    
    return trans_multi


# ***function: genTrans_tele
# this function takes in a set of teleport nodes and compute the 
# transition matrix accordingly. the difference between genTrans function
# is that genTrans produces transition matrices for single source nodes,
# while genTrans_tele produces transition matrices for a set of teleport
# nodes.
def genTrans_tele(nnodes, g, features, tele, alpha, beta):
    # feature is supplied in per-edge manner
    # the transition matrix is created with teleportation
    trans = np.zeros((nnodes, nnodes))
    for i in range(len(g)):
        strength = calStrength(features[g[i][0]][g[i][1]], beta)
        trans[g[i][0], g[i][1]] = strength
        trans[g[i][1], g[i][0]] = strength
    
    # normalize the transition matrix
    for i in range(nnodes):
        tempSum = sum(trans[i,])
        if tempSum > 0:
            trans[i,] = map(lambda x: x/tempSum, trans[i, ])
    
    # create a list of transition matrices for a set of teleport sets
    trans_multi = []
    
    for t in range(len(tele)):
        teleSize = len(tele[t])
        one = np.zeros((nnodes, nnodes))
        if teleSize > 0:
            for i in range(nnodes):
                for j in range(teleSize):
                    one[i, tele[t][j]] = 1.0/teleSize
            # the calculated transition matrix with teleportation
            trans_multi.append((1-alpha)*trans + alpha*one)
        else:
            # transition matrix without transporation since the 
            # input teleport set is empty
            trans_multi.append(trans)
    
    return trans_multi

# ***function: genFeatures
# input features are in edge list form, this function transfer the features into
# matrix-style, with each element in the matrix a the feature vecotr of 
# corresponding edge, the element is [] if the particular edge does not exist
def genFeatures(nnodes, g, features):
    # very elemnt in the array is a list
    fea = [[ [] for x in range(nnodes) ] for x in range(nnodes) ]
    # create a feature matrix
    for i in range(len(g)):
        fea[g[i][0]][g[i][1]] = features[i]
        fea[g[i][1]][g[i][0]] = features[i]
    
    return fea


############################################
############################################
## Below are the functions for learning process

# ***function: iterPageDiff
# this function use power-iteration-like method to return the gradient of 
# Supervised Random Walk pagerank scores
def iterPageDiff(pdiff, p, trans, transdiff):
    pdiffnew = np.dot(pdiff, trans) + np.dot(p, transdiff)
    while not(np.allclose(pdiff, pdiffnew)):
        pdiff = pdiffnew
        pdiffnew = np.dot(pdiff, trans) + np.dot(p, transdiff)
    return pdiffnew[0]


#########################################
### this function is not used anymore ###
#########################################
# ***function: diffQelem
# this function is called by diffQ, return the (i, j)-th element of the 
# derivative of transition matrix with respect to k-th element of beta
def diffQelem(features, beta, trans_p, alpha, row, col, k):
    # calculates the element value of transition matrix's differentiation
    # first calculate the denominator part    
    denom = 0
    xdenom = 0
    for j in range(int(np.shape(trans_p)[1])):
        if trans_p[row, j] > 0:
            # should check on the original version of transition matrix, 
            # because teleportation does not contribute to gradient
            #temp = calStrength(np.asarray(features[row,])*np.asarray(features[j,])
            #, beta)
            temp = calStrength(features[row][j], beta)
            denom += temp
            #xdenom += (np.asarray(features[row,])*np.asarray(features[j,]))[k] * temp
            xdenom += features[row][j][k] * temp
    #curFeat = np.asarray(features[row,])*np.asarray(features[col,])
    curFeat = features[row][col]
    strength = calStrength(curFeat, beta)
    
    elem = (1-alpha)*(curFeat[k]*strength*denom - strength*xdenom) / (denom**2)
    
    return elem


# ***function: diffQ
# given a Supervised Random Walk transition matrix, return the derivative of 
# transition matrix with respect to the k-th element in parameter beta
def diffQ(features, beta, trans_p, alpha):
    
    nnodes = int(np.shape(trans_p)[0])
	
    # first compute the (unnormalized) edge strength matrix and the gradient matrix
    sMat = np.zeros((nnodes, nnodes))
    for i in range(int(np.shape(trans_p)[0])):
        for j in range(i, int(np.shape(trans_p)[1])):
            if trans_p[i, j] > 0:
                strength = calStrength(features[i][j], beta)
                sMat[i, j] = strength
                sMat[j, i] = strength
    
    # gradQ is the gradient of strength matrix
    # a list of matrices is computed, with the k-th item in the list be the 
    # derivative of strength matrix with respect to the k-th element in beta
    # vecotr
    gradS = []
    for i in range(len(beta)):
        gradS.append(np.zeros((nnodes, nnodes)))    
    for i in range(int(np.shape(trans_p)[0])):
        for j in range(int(np.shape(trans_p)[1])):
            if trans_p[i, j] > 0:
                gradTemp = strengthDiff(features[i][j], beta)
                for k in range(len(beta)):
                    gradS[k][i, j] = gradTemp[k]
    
    
    # compute the gradient of transition matrix
    # a list of matrices is computed, with k-th element in the list be 
    # the derivative of transition matrix with respect to the k-th 
    # element in parameter vecotor beta
    qp = []
    for i in range(len(beta)):
        qp.append(np.zeros((nnodes, nnodes)))
    
    for i in range(int(np.shape(trans_p)[0])):
        # for each row in the gradient matrix, some common factors can be 
        # computed first
        sumStrength = 0
        sumDiff = [0] * len(beta)
        for j in range(int(np.shape(trans_p)[1])):
            if trans_p[i, j] > 0:
                sumStrength += sMat[i, j]
                for k in range(len(beta)):
                    sumDiff[k] += gradS[k][i, j]
        # individual entries can then be computed
        for j in range(int(np.shape(trans_p)[1])):
            if trans_p[i, j] > 0:
                for k in range(len(beta)):
                    qp[k][i, j] = (sumStrength ** -2)*( gradS[k][i, j]*sumStrength -
                    sMat[i, j]*sumDiff[k])*(1 - alpha)
        
    return qp


# ***function: costFunc
# this is the cost function in learning process
# given the page rank socres of two nodes, one in future link set
# one in no-link set, and an offset parameter, the cost function value is 
# returned. (return value is a scalar)
def costFunc(pl, pd, offset):
    #return (max(0, pl - pd + offset))**2
    return 1.0 / (1 + np.exp(-1.0 * (pl - pd)/offset))


# ***function: costDiff
# this is the gradient of cost function
# given the page rank socres of two nodes, one in future link set
# one in no-link set, and an offset parameter, the gradient of cost function 
# is returned. (return value is a scalar)
def costDiff(pl, pd, offset):
    #return 2.0*(max(0, pl - pd + offset))
    return (1.0 / offset) * np.exp(-1.0 * (pl - pd)/offset) * (costFunc(pl, pd, offset) ** 2)


# ***function: minObj
# this is the object function to be minimized in the learning process
# the future link set and no-link from training set should be given, also, 
# parameters in the learning process and the graph of training set should be 
# given.
# Supervised Random Walk pagerank scores are calculated for each node based on 
# input training graph and features, then cost function value is calculated 
# according to the derived pagerank scores.
# (return value is a scalar)
def minObj(Dset, Lset, offset, lam, nnodes, g, features, source, alpha, beta):
    # calculate PageRank according to features and beta values
    
    # transform input features into matrix form
    features_m = genFeatures(nnodes, g, features)
    
    # compute transition matrices for sources
    trans = genTrans(nnodes, g, features_m, source, alpha, beta)
    
    # cost value    
    cost = 0
    # calculate cost function for every selected sources nodes
    for i in range(len(source)):
        pp = np.repeat(1.0/nnodes, nnodes)
        pgrank = iterPageRank(pp, trans[i])
        
        # compute cost from the generated PageRank value    
        for d in Dset[i]:
            for l in Lset[i]:
                cost += costFunc(pgrank[l], pgrank[d], offset)
    
    penalty = lam * np.dot(beta, beta)
    
    return (cost + penalty)


# ***function: objDiff
# this is the gradient of the object function
# given training graph, training sets and parameters, gradient of the object 
# function is returned. This is required in gradient descent and BFGS optimization
# processes.
# Supervised Random Walk pagerank is calculated then served as a basis to compute
# the gradient of pagerank scores. Gradient of pagerank scores are derived by 
# power-iteration-like method.
# (return value is a vector with the dimension of parameter beta)
def objDiff(Dset, Lset, offset, lam, nnodes, g, features, source, alpha, beta):
    diffVec = [0] * len(beta)
    # calculate PageRank according to features and beta values
    
    # transform input features into matrix form
    features_m = genFeatures(nnodes, g, features)        
    
    ###########################################################
    ### trans_p and transDiff are independent of source node ##
    ###########################################################
    # trans_p is the original transition matrix 
    # (without teleportation and varying strength)
    # this is used to calculate gradient of transition matrix
    trans_p = genTrans_plain(nnodes, g, [0], 0)[0]
        
    # a list of matrices is returned by diffQ function
    transDiff = diffQ(features_m, beta, trans_p, alpha)
    ###########################################################
    ###########################################################
    
    # compute transition matrices for sources
    trans = genTrans(nnodes, g, features_m, source, alpha, beta)
    
    # calculate gradient for every selected sources nodes
    for i in range(len(source)):
    
        pp = np.repeat(1.0/nnodes, nnodes)
        pgrank = iterPageRank(pp, trans[i])
        
        for k in range(len(beta)):
            tempObjDiff = 0
            pDiff = np.zeros((1, nnodes))
            pDiff = iterPageDiff(pDiff, pgrank, trans[i], transDiff[k])
            for d in Dset[i]:
                for l in Lset[i]:
                    tempObjDiff += costDiff(pgrank[l], pgrank[d], offset)*(pDiff[l] - pDiff[d])
            # penalty term
            #tempObjDiff += 2.0 * lam * beta[k]
            
            #diffVec.append(tempObjDiff)
            diffVec[k] += tempObjDiff
    
    # penalty term
    for k in range(len(beta)):
        diffVec[k] += 2.0 * lam * beta[k]
    
    return np.asarray(diffVec)

        
# ***function: trainModel
# users call this function to train beta parameter of Supervised Random Walk algorithm
# a training set and training graph must be specified as well as the parameters for the 
# learning process. Also, initial guess of beta parameter shall be given
# scipy's L-BFGS-B optimizer is called to iteratively optimize the object function,
# object function and the gradient of cost function is the main input to BFGS
# optimizer
def trainModel(Dset, Lset, offset, lam, nnodes, g, features, source, alpha, beta_init):
    #beta_Opt = fmin_bfgs(functools.partial(minObj, Dset, Lset, 0, 0, nnodes, g, features, 
    #                        source, alpha), beta_init, fprime = functools.partial(objDiff, 
    #                        Dset, Lset, 0, 0, nnodes, g, features, source, alpha))
    
    beta_Opt = fmin_l_bfgs_b(functools.partial(minObj, Dset, Lset, offset, lam, nnodes, g, features, 
                            source, alpha), beta_init, fprime = functools.partial(objDiff, 
                            Dset, Lset, offset, lam, nnodes, g, features, source, alpha))
    
    return beta_Opt


############################################
############################################

In [3]:
import json

print "Reading data..."

# load the sanpshots of 6-30 and 12-31,
# 6-30 is a graph used as a basis of the learning process
# 12-31 provides both training data and test data
fp = open('amazon-meta_item_item_1.txt', 'r')
fp_end = open('amazon-meta_item_item_2.txt', 'r')

nnodes = 1099
degrees = [0] * nnodes
edges = []
edges_end = []
features = [[], []]
features_end = [[], []]
for line in fp:
    temp = line.strip().split(',')
    edges.append((int(temp[0]), int(temp[1])))
    features[0].append(float(temp[2]))
    features[1].append(float(temp[3]))
    degrees[int(temp[0])] += 1
    degrees[int(temp[1])] += 1

for line in fp_end:
    temp = line.strip().split(',')
    edges_end.append((int(temp[0]), int(temp[1])))
    features_end[0].append(float(temp[2]))
    features_end[1].append(float(temp[3]))

fp.close()
fp_end.close()

# normalize features
u0 = np.mean(features[0])
u1 = np.mean(features[1])
s0 = np.std(features[0])
s1 = np.std(features[1])

features[0] = map(lambda x: (x-u0)/s0, features[0])
features[1] = map(lambda x: (x-u1)/s1, features[1])

edge_feature = []
# features are formed with intercept term
for i in range(len(edges)):
    edge_feature.append([features[0][i], features[1][i]])


#######################################
#### Training set formation ###########
#######################################

print "Forming training set..."

# compute the candidate set for future links according to source node
# train model with a set of source nodes
elig_source = []
for i in range(len(degrees)):
    if degrees[i] > 0:
        elig_source.append(i)

# pick nodes with number of future links larger than theshold
# these nodes then served as either training node or test node
Dsize_cut = 0

D_source = []
Dset_all = []
Lset_all = []
for i in range(len(elig_source)):
    sNeighbor = []
    for e in edges:
        if e[0] == elig_source[i]:
            sNeighbor.append(e[1])
        elif e[1] == elig_source[i]:
            sNeighbor.append(e[0])
    candidates = list(set(list(range(nnodes))) - set([elig_source[i]]) - set(sNeighbor))
    
    sNeighbor_end = []
    for e in edges_end:
        if e[0] == elig_source[i]:
            sNeighbor_end.append(e[1])
        elif e[1] == elig_source[i]:
            sNeighbor_end.append(e[0])
    tempDset = list(set(sNeighbor_end) - set(sNeighbor))
    if len(tempDset) >= Dsize_cut:
        tempLset = list(set(candidates) - set(tempDset))
        Dset_all.append(tempDset)
        Lset_all.append(tempLset)
        D_source.append(elig_source[i])

# randomly pick nodes with current degree > 0 and number of future 
# links >= Dsize_cut as the training set
trainSize = 100
testSize = 50
# this index is the index of source nodes in D_source list
source_index = np.random.choice(list(range(len(D_source))), 
                                size=trainSize, replace=False)
source = []
Dset = []
Lset = []
for i in source_index:
    source.append(D_source[i])
    Dset.append(Dset_all[i])
    Lset.append(Lset_all[i])

Reading data...
Forming training set...


In [10]:
test_index = np.random.choice(list(set(list(range(len(D_source)))) - 
set(source_index)), size=testSize, replace=False)



testSet = []
Dset_test = []
Lset_test = []
candidates_test = []


# original code

for i in test_index:
    testSet.append(D_source[i])
    Dset_test.append(Dset_all[i])
    Lset_test.append(Lset_all[i])
    candidates_test.append(Dset_all[i] + Lset_all[i])



#######################################
#### Model training phase #############
#######################################

print "Training model..."

# set up parameters
lam = 50
offset = 0.01
alpha = 0.3
beta_init = [0.5, 0.5]

#ff = genFeatures(nnodes, edges, edge_feature)
#trans_p = genTrans_plain(nnodes, edges, 0, 0)
#qqp = diffQ(ff, [0, 0.5, 0.5], trans_p, alpha)
#print qqp
beta_Opt = trainModel(Dset, Lset, offset, lam, nnodes, edges, edge_feature, 
                      source, alpha, beta_init)

# train model direclty wtth test set, compare performance with UWRW
#beta_Opt = trainModel(Dset_test, Lset_test, offset, lam, nnodes, edges, edge_feature, 
#                      testSet, alpha, beta_init)

print "Training source set:\n", source
print "\nTrained model parameters:\n", beta_Opt


#######################################
#### Test model performance ###########
#######################################

print "Evaluating model performance..."

# link prediction with transition matrices computed with trained parameters
ff = genFeatures(nnodes, edges, edge_feature)
trans_srw = genTrans(nnodes, edges, ff, testSet, alpha, beta_Opt[0])
#trans_srw = genTrans(nnodes, edges, ff, testSet, alpha, [10, 10])

# compute personalized PageRank for test nodes to recommend links
pgrank_srw = []
cand_pairs_srw = []
link_hits_srw = []
for i in range(len(testSet)):
    pp = np.repeat(1.0/nnodes, nnodes)
    curpgrank = iterPageRank(pp, trans_srw[i])
    # record the pgrank score
    pgrank_srw.append(curpgrank)
    
    # find the top ranking nodes in candidates set
    cand_pairs = []
    for j in candidates_test[i]:
        cand_pairs.append((j, curpgrank[j]))
    cand_pairs = sorted(cand_pairs, key = lambda x: x[1], reverse=True)
    # record candidate-pagerank pairs
    cand_pairs_srw.append(cand_pairs)
    
    # calculate precision of the top-Dsize_cut predicted links
    link_hits = 0    
    for j in range(5):
        if cand_pairs[j][0] in Dset_test[i]:
            link_hits += 1
    link_hits_srw.append(link_hits)

print "\nSRW performance: ", np.mean(link_hits_srw)

Training model...
Training source set:
[946, 526, 279, 693, 575, 31, 573, 713, 984, 703, 449, 1027, 669, 408, 911, 144, 1066, 705, 2, 206, 335, 236, 1007, 386, 330, 443, 115, 1021, 193, 820, 1056, 1092, 845, 800, 552, 308, 504, 885, 690, 938, 741, 574, 999, 822, 647, 980, 1005, 940, 515, 174, 106, 58, 1074, 1097, 20, 812, 441, 877, 773, 214, 1040, 840, 762, 199, 391, 600, 548, 366, 430, 1035, 740, 867, 742, 738, 707, 748, 1088, 907, 847, 492, 316, 965, 774, 631, 312, 961, 949, 184, 148, 852, 1083, 1087, 28, 973, 682, 350, 141, 882, 82, 417]

Trained model parameters:
(array([ 0.68950976,  0.51500747]), 16958.397390858812, {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.09763054, -0.13898487]), 'nit': 3, 'funcalls': 45})
Evaluating model performance...

SRW performance:  0.22


In [12]:
total = 0
for i in range(len(testSet)):
    total += len(Dset_test[i])
print sum(link_hits_srw)*1.0/total

50
